# San Francisco Housing Cost Analysis

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
from dotenv import load_dotenv

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api=os.getenv('mapbox')

## Load Data

In [ ]:
# Read the census data into a Pandas DataFrame with year as the index
file_path=Path('Data/sfo_neighborhoods_census_data.csv')
sfo_data_year_index=pd.read_csv(file_path,index_col='year')
sfo_data_year_index.head()

## Housing Units Per Year
In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function.
**Hint:** Use the Pandas `groupby` function.
**Optional challenge:** Use the min, max, and std to scale the y limits of the chart.

In [ ]:
# Calculate the mean number of housing units per year (hint: use groupby)
housing_units_per_year=sfo_data_year_index.groupby('year').mean().drop(columns=['sale_price_sqr_foot','gross_rent'])
housing_units_per_year

In [ ]:
# Save the dataframe as a csv file
housing_units_per_year.to_csv(r'avg_units_by_year.csv',index=True)  

In [ ]:
# Use Pandas plot function to plot the average housing units per year.
### Note: You will need to manually adjust the y limit of the chart using the min and max values from above.
housing_units_per_year.plot(kind='bar',
                           ylim=(370000,385000),
                           legend=False,
                           title='Housing Units in San Francisco from 2010 to 2016'
                          )
# Optional - Use min, max, and std to scale the y limits of the chart


- - - 

## Average Housing Costs in San Francisco Per Year

In this section, you will calculate the average monthly rent and the average price per square foot for each year. An investor may wish to better understand the sales price of the rental property over time. For example, a customer will want to know if they should expect an increase or decrease in the property value over time so they can determine how long to hold the rental property.  Plot the results as two line charts.

**Optional challenge:** Plot each line chart in a different color.

In [ ]:
# Calculate the average sale price per square foot and average gross rent
avg_price_sqft_rent=sfo_data_year_index.groupby('year').mean().drop(columns=['housing_units'])
display(avg_price_sqft_rent)

In [ ]:
# Create two line charts, one to plot the average sale price per square foot and another for average montly rent
## Line chart for average sale price per square foot
avg_price_sqr_foot=pd.DataFrame(avg_price_sqft_rent,columns=['sale_price_sqr_foot'])
avg_price_sqr_foot.plot(
    legend=False,
    color='purple',
    title ='Average Price per SqFt by Year'
)
## Line chart for average montly rent
avg_rent=pd.DataFrame(avg_price_sqft_rent,columns=['gross_rent'])
avg_rent.plot(
    legend=False,
    color='red',
    title='Average Gross Rent by Year'
)

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create two interactive visulizations of average prices with a dropdown selector for the neighborhood. The first visualization will be a line plot showing the trend of average price per square foot over time for each neighborhood.  The second will be a line plot showing the trend of average monthly rent over time for each neighborhood.

**Hint:** It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [ ]:
## Read the census data into a Pandas DataFrame without year as the index
sfo_data=pd.read_csv(file_path)
# Group by year and neighborhood and new dataframe with mean values
grp_year_neighborhoods=sfo_data.groupby(['year','neighborhood'],as_index=False).mean()
### Print San Francisco census dataframe without year index ###
sfo_data.head(10)

In [ ]:
## Use hvplot to create an interactive line chart of the average price per sq ft.
# The plot should have a dropdown selector for the neighborhood
grp_year_neighborhoods.hvplot(
    x='year',
    y='sale_price_sqr_foot',
    xlabel='Year',
    ylabel='Avg Sale Price per Square Foot',
    groupby='neighborhood'
)

In [ ]:
# Use hvplot to create an interactive line chart of the average monthly rent.
# The plot should have a dropdown selector for the neighborhood
grp_year_neighborhoods.hvplot(
    x='year',
    y='gross_rent',
    xlabel='Year',
    ylabel='Avg Monthly Rent',
    groupby='neighborhood'
)

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price per square foot for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [ ]:
# Getting the data from the top 10 expensive neighborhoods to own
top10_exp_hoods=sfo_data.groupby(['neighborhood'],as_index=False).mean().nlargest(10,'sale_price_sqr_foot').drop(columns='year')
# top10_exp_hoods=top10_exp_hoods.drop
top10_exp_hoods

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
top10_exp_hoods.hvplot.bar(
    x='neighborhood',
    y='sale_price_sqr_foot',
    xlabel='Neighborhood',
    ylabel='Avg Sale Price Per Square Foot',
    legend=False,
    rot=90,
    title='Top 10 Most Expensive Neighborhoods in San Francisco',
    width=1000,
    height=500,
)

- - - 

## Comparing cost to purchase versus rental income

In this section, you will use `hvplot` to create an interactive visualization with a dropdown selector for the neighborhood. This visualization will feature a side-by-side comparison of average price per square foot versus average montly rent by year.  

**Hint:** Use the `hvplot` parameter, `groupby`, to create a dropdown selector for the neighborhood.

In [ ]:
# Fetch the previously generated DataFrame that was grouped by year and neighborhood
grp_year_neighborhoods.head(10)

In [ ]:
# Plotting the data from the top 10 expensive neighborhoods
grp_year_neighborhoods.hvplot.bar(
    x='year',
    y=['gross_rent','sale_price_sqr_foot'],
    xlabel='Neighborhood',
    ylabel='Values',
    legend=False,
    rot=90,
    height=500,
    groupby='neighborhood',
    title='Rent and Price Per Square Foot in Each Neighborhood Per Year',
    width=1000,
)

- - - 

## Neighborhood Map

In this section, you will read in neighborhoods location data and build an interactive map with the average house value per neighborhood. Use a `scatter_mapbox` from Plotly express to create the visualization. Remember, you will need your Mapbox API key for this.

### Load Location Data

In [ ]:
# Load neighborhoods coordinates data
coordinates_df=pd.read_csv(Path('Data/neighborhoods_coordinates.csv'))
coordinates_df.head()

### Data Preparation

You will need to join the location data with the mean values per neighborhood.

1. Calculate the mean values for each neighborhood.

2. Join the average values with the neighborhood locations.

In [ ]:
# Calculate the mean values for each neighborhood
hood_value_df=sfo_data.drop(columns=['year']).rename(columns={'neighborhood':'Neighborhood'}).groupby(['Neighborhood']).mean()
hood_value_df.head()

In [ ]:
# Join the average values with the neighborhood locations
hood_location_values_df=pd.merge(coordinates_df,hood_value_df,on='Neighborhood')
hood_location_values_df.head()

### Mapbox Visualization

Plot the average values per neighborhood using a Plotly express `scatter_mapbox` visualization.

In [ ]:
# Set the mapbox access token
px.set_mapbox_access_token(map_box_api)
# Create a scatter mapbox to analyze neighborhood info
hood_location_values=px.scatter_mapbox(
    hood_location_values_df,
    lat='Lat',
    lon='Lon',
    color='gross_rent',
    color_continuous_scale=px.colors.cyclical.IceFire,
    title='Average Sale Price Per Square Foot and Gross Rent In San Francisco',
    size='sale_price_sqr_foot',
    zoom=10,
    width=1000,
    height=600
)
hood_location_values.show()

## Cost Analysis - Optional Challenge

In this section, you will use Plotly express to create visualizations that investors can use to interactively filter and explore various factors related to the house value of the San Francisco's neighborhoods. 

### Create a DataFrame showing the most expensive neighborhoods in San Francisco by year

In [ ]:
# Fetch the data from all expensive neighborhoods per year.
most_exp_hoods_per_year=grp_year_neighborhoods[grp_year_neighborhoods['neighborhood'].isin(top10_exp_hoods['neighborhood'])]
most_exp_hoods_per_year.head()

### Create a parallel coordinates plot and parallel categories plot of most expensive neighborhoods in San Francisco per year


In [ ]:
# Parallel Categories Plot
px.parallel_categories(
    top10_exp_hoods,
    color='sale_price_sqr_foot',
    color_continuous_scale=px.colors.sequential.Inferno,
    dimensions=['neighborhood','sale_price_sqr_foot','housing_units','gross_rent'],
    labels={'neighborhood':'Neighborhood','sale_price_sqr_foot':'Sale Price per Square Foot','housing_units':'Housing Units','gross_rent':'Gross Rent'},
    width=1000,
)

In [ ]:
# Parallel Coordinates Plot
px.parallel_coordinates(
    top10_exp_hoods,
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    dimensions=['sale_price_sqr_foot','housing_units','gross_rent'],
    labels={'sale_price_sqr_foot':'Sale Price per Square Foot','housing_units':'Housing Units','gross_rent':'Gross Rent'},
    width=1000,
) 

### Create a sunburst chart to conduct a costs analysis of most expensive neighborhoods in San Francisco per year

In [ ]:
# Sunburst Plot
px.sunburst(
    most_exp_hoods_per_year,
    path=['year','neighborhood'],
    color='gross_rent',
    color_continuous_scale=px.colors.sequential.Blues,
    values='sale_price_sqr_foot',
    width=1000,
    height=1000
    )